In [1]:
from pynq import Overlay
from pynq import allocate 
import numpy as np
import struct
import subprocess
import time
import re

In [2]:
#PARAMETERS
MAX_ROT_GROUP_SIZE = 128
MAX_KSI_POWS_SIZE = 513
MAX_QPOW_VEC_SIZE = 65
SIGMA = 3.2
H = 64 
RND_SIZE = 257
LOG_N = 8
LOG_Q = 32
LOG_q = 32
LOG_P = 30
MAX_Z_SIZE = 256
MAX_ZZX_DEGREE = 513
MAX_SLOTS = 8
slots = np.int64(8)

In [3]:
secret_key_coeff_filename = "data/secretKey.data"
secret_key_degree_filename = "data/secretKey.degree"

message_filename = "data/message.data"

cipher_ax_coeff_filename = "data/cipherAx.data"
cipher_ax_degree_filename = "data/cipherAx.degree"

cipher_bx_coeff_filename = "data/cipherBx.data"
cipher_bx_degree_filename = "data/cipherBx.degree"

decrypted_message_filename = "data/decrypted.data"

data_generator_path = './data_gen'

In [4]:
# Execute the binary for data generation
result = subprocess.run([data_generator_path], capture_output=True, text=True)

# Check the return code to see if the execution was successful
if result.returncode == 0:
    # Print the output of the binary
    print("Output stdout:", result.stdout)
else:
    # Print the error if the binary was not executed successfully
    print("Error executing the binary")
    print("Output stderr:", result.stderr)

output = result.stdout
encryption_time_match = re.search(r'\[INFO\] SW encryption execution time: (\d+) microseconds', output)
decryption_time_match = re.search(r'\[INFO\] SW decryption execution time: (\d+) microseconds', output)

if encryption_time_match and decryption_time_match:
    encryption_time = int(encryption_time_match.group(1))
    decryption_time = int(decryption_time_match.group(1))

Output stdout: context initiazlied
Context Parameters : 
sigma: 3.2
h: 64
N: 256
Nh: 128
logNh: 7
M: 512
logQQ: 64
Q: 4294967296
QQ: 18446744073709551616
secret key initialized
scheme initialized
[INFO] SW encryption execution time: 1051864 microseconds
[INFO] SW decryption execution time: 507172 microseconds



In [5]:
def read_int64_from_file(filename):
    # Reads a single 64-bit integer from the binary file
    data = np.fromfile(filename, dtype=np.int64, count=1)
    
    # Checks if the file contains at least one 64-bit integer
    if data.size == 0:
        raise ValueError(f"The file {filename} is empty or does not contain a valid 64-bit integer.")
    
    return data[0]

In [6]:
def read_and_convert_array(filename, size, max_z_size):
    # Calculates the number of int64 needed to represent an integer of max_z_size bits
    num_int64 = max_z_size // 64
    
    # Reads the data from the binary file as integers of max_z_size bits
    data = np.fromfile(filename, dtype=np.int64, count=size*num_int64)
    
    return data

In [7]:
def read_complex_array_from_file(filename, size):
    # Reads the data from the binary file as 128-bit complex numbers (complex128)
    data = np.fromfile(filename, dtype=np.complex128, count=size)

    return data

In [8]:
sx_degree = read_int64_from_file(secret_key_degree_filename)
print(f"Secret key degree: {sx_degree}")

Secret key degree: 256


In [9]:
sx = read_and_convert_array(secret_key_coeff_filename, sx_degree +1, MAX_Z_SIZE)
print(f"Size of the secret key in bytes: {sx.nbytes}")


Size of the secret key in bytes: 8224


In [10]:
ci_ax_degree = read_int64_from_file(cipher_ax_degree_filename)
print(f"Cipher.ax degree: {ci_ax_degree}")

ci_ax = read_and_convert_array(cipher_ax_coeff_filename, ci_ax_degree + 1, MAX_Z_SIZE)
print(f"Size of Cipher.ax in bytes: {ci_ax.nbytes}")


Cipher.ax degree: 256
Size of Cipher.ax in bytes: 8224


In [11]:
ci_bx_degree = read_int64_from_file(cipher_bx_degree_filename)
print(f"Cipher.bx degree: {ci_bx_degree}")

ci_bx = read_and_convert_array(cipher_bx_coeff_filename, ci_bx_degree + 1, MAX_Z_SIZE)
print(f"Size of Cipher.bx in bytes: {ci_bx.nbytes}")


Cipher.bx degree: 256
Size of Cipher.bx in bytes: 8224


In [12]:
message_array = read_complex_array_from_file(message_filename, slots)
print(f"Size of the message array in bytes: {message_array.nbytes}")
print("Message Array:", message_array)

Size of the message array in bytes: 128
Message Array: [0.24428 +0.651282j 0.868426+0.678266j 0.777887+0.301179j
 0.883243+0.13489j  0.991355+0.510088j 0.710801+0.774569j
 0.809488+0.386638j 0.609341+0.486097j]


In [13]:
overlay = Overlay("./ckks_decryption_wrapper1.xsa")

In [14]:
dec_ip = overlay.ckks_dec_ker_0

In [15]:
buf_sx = allocate((sx_degree+1)*(MAX_Z_SIZE//64), np.int64)

buf_sx[:] = sx[:]
buf_sx.flush()

buf_ci_ax = allocate((ci_ax_degree+1)*(MAX_Z_SIZE//64), np.int64)

buf_ci_ax[:] = ci_ax[:]
buf_ci_ax.flush()

buf_ci_bx = allocate((ci_bx_degree+1)*(MAX_Z_SIZE//64), np.int64)
buf_ci_bx[:] = ci_bx[:]
buf_ci_bx.flush()

buf_message = allocate(MAX_SLOTS, np.complex128)

In [16]:
slots_bytes = slots.tobytes()
sx_degree_b = sx_degree.tobytes()
ci_ax_degree_b = ci_ax_degree.tobytes()
ci_bx_degree_b = ci_bx_degree.tobytes()

In [17]:
dec_ip.write(0x10,buf_sx.physical_address)
dec_ip.write(0x1c,sx_degree_b[:4])
dec_ip.write(0x20,sx_degree_b[-4:])
dec_ip.write(0x28,buf_ci_ax.physical_address)
dec_ip.write(0x34,buf_ci_bx.physical_address)
dec_ip.write(0x40,ci_ax_degree_b[:4])
dec_ip.write(0x44,ci_ax_degree_b[-4:])
dec_ip.write(0x4c,ci_bx_degree_b[:4])
dec_ip.write(0x50,ci_bx_degree_b[-4:])
dec_ip.write(0x58,slots_bytes[:4])
dec_ip.write(0x5c,slots_bytes[-4:])
dec_ip.write(0x64,buf_message.physical_address)

In [18]:
dec_ip.write(0x00,1)
start_time = time.perf_counter()
while(dec_ip.read(0x00) & 0x04 != 0x04):
      pass
end_time = time.perf_counter()


In [19]:
# Calculate the execution time in microseconds
execution_time_us = (end_time - start_time) * 1_000_000


In [20]:
buf_message.invalidate()

In [21]:
decrypted_message = read_complex_array_from_file(decrypted_message_filename, slots)
print(f"Size of the decrypted message in bytes: {decrypted_message.nbytes}")


Size of the decrypted message in bytes: 128


In [22]:
for i in range(0, slots):
    # Round both values to 15 decimal places
    buf_val = np.round(buf_message[i], 15)
    decrypted_val = np.round(decrypted_message[i], 15)
    
    if buf_val != decrypted_val:
        print("ERROR HW DECRYPTED MESSAGE IS DIFFERENT AT INDEX: ",i," ERROR: ",buf_val - decrypted_val)

In [23]:
for i in range (0, slots):
    if(buf_message[i]!=message_array[i]):
        print("Noise Detected during decryption: ~~ ",buf_message[i]-message_array[i])

Noise Detected during decryption: ~~  (-8.115658700624806e-10-5.378828160651494e-09j)
Noise Detected during decryption: ~~  (3.168058038127697e-10-1.4927411529086498e-09j)
Noise Detected during decryption: ~~  (4.247935336820774e-12+4.866468339415064e-10j)
Noise Detected during decryption: ~~  (-1.405039196100688e-09+1.5683042642322675e-09j)
Noise Detected during decryption: ~~  (-4.088327454354612e-10-1.0848363318771703e-09j)
Noise Detected during decryption: ~~  (-1.5094723249120534e-09+8.10279843221906e-10j)
Noise Detected during decryption: ~~  (5.772484712451842e-10-1.3640383267343736e-10j)
Noise Detected during decryption: ~~  (-1.1345583361688227e-09-1.5044082091186795e-09j)


In [24]:
print("HW EXECUTION TIME: ", execution_time_us, " microseconds")
print("SW EXECUTION TIME: ", encryption_time, " microseconds")
print("HW SPEEDUP: ", encryption_time/execution_time_us, "x")

HW EXECUTION TIME:  400.46684443950653  microseconds
SW EXECUTION TIME:  1051864  microseconds
HW SPEEDUP:  2626.594472439258 x
